In [ ]:
# Author : Purushothaman Natarajan
# Title : Cancer prediction using CNN (transfer learning)
# Dataset : Open source dataset found in Kaggle (Oral Cancer)

In [ ]:
# Install necessary Libraries
!pip install tensorflow
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install tabulate
!pip install seaborn

In [ ]:
from PIL import Image
import os

In [ ]:
#print the current directory
print(os.getcwd())

In [ ]:
images = []
labels = []

master_data_path="/tf/red-model/C_Devel/unzipps/Cancer Prediction"

def load_images_from_folder(folder_path, label):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isdir(file_path):
            # If the item is a directory, recursively load images from it
            load_images_from_folder(file_path, filename)
        elif filename.lower().endswith(('png', 'jpg', 'jpeg')):
            # If the item is an image file, load it and add it to the images list
            try:
                img = Image.open(file_path)
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f'Error loading image {file_path}: {str(e)}')
        else:
            print(f'Skipping non-image file: {file_path}')

# Iterate through folders in the master folder
for folder_name in os.listdir(master_data_path):
    folder_path = os.path.join(master_data_path, folder_name)
    if os.path.isdir(folder_path):
        # Load images and labels from the current folder
        load_images_from_folder(folder_path, folder_name)

print(f'Images loaded: {len(images)}')
print(f'Labels loaded: {len(labels)}')

In [ ]:
#counting the number of labels in each classes

count_of_classes = {}

for label in labels:
    if label in count_of_classes:
        count_of_classes[label] +=1
    else:
        count_of_classes[label] = 1


for key, value in count_of_classes.items():
    print(f'{key}:{value}')

In [ ]:
dimension = []
for idx, image in enumerate(images):
    width, height = image.size
    current_dimension = (width, height)
    dimension.append(current_dimension)

unique_dimension = list(set(dimension))
print(f'Unique dimension in the dataset: {len(unique_dimension)}')

In [ ]:
import numpy as np

In [ ]:
# Convert PIL images to numpy arrays
numpy_images = [np.array(image) for image in images]

# Resize and convert images to RGB format if necessary
target_size = (224, 224)
reshaped_images = []
for idx, image in enumerate(numpy_images):
    pil_image = Image.fromarray(image)
    pil_image = pil_image.resize(target_size)
    # Convert to RGB if image is grayscale
    if pil_image.mode != 'RGB':
        pil_image = pil_image.convert('RGB')
    reshaped_images.append(np.array(pil_image))


print("Dimension of reshaped_images:", reshaped_images[0].shape)

print("Data type of reshaped_images:", reshaped_images[0].dtype)


# Convert images to float32 and normalize to [0, 1]
normalized_images = np.array(reshaped_images, dtype=np.float32) / 255.0

# Verify the shape and data type of processed_images
print("Shape of normalized_images:", normalized_images.shape)
print("Data type of processed_images:", normalized_images.dtype)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(labels[0])
plt.imshow(normalized_images[0])
plt.show()

## Handling the imbalance in the dataset

In [ ]:
#counting the number of labels in each classes

count_of_classes = {}

for label in labels:
    if label in count_of_classes:
        count_of_classes[label] +=1
    else:
        count_of_classes[label] = 1


for key, value in count_of_classes.items():
    print(f'{key}:{value}')

In [ ]:
required_size = round(len(labels)/2)
print(f"we should have {required_size} images in each class")

In [ ]:
set(labels)

In [ ]:
import random
import numpy as np

# Define the number of synthetic images you want to generate for each class
num_synthetic_images_per_class = 638

# Define a function to apply augmentations and generate synthetic images
def generate_synthetic_image(image_data, label):
    # Apply random augmentations based on your requirements
    # For example, you can resize, rotate, flip, change brightness, etc.
    # Here, I'm applying random horizontal flip and rotation
    augmented_image = image_data.copy()
    if random.choice([True, False]):
        augmented_image = np.fliplr(augmented_image)
    rotation_angle = random.randint(-45, 45)
    augmented_image = np.rot90(augmented_image, k=random.randint(0, 3), axes=(0, 1))

    # Return the augmented image and its label
    return augmented_image, label

# Separate images and labels by class
normal_images = [image_data for image_data, label in zip(normalized_images, labels) if label == 'Normal images']
oscc_images = [image_data for image_data, label in zip(normalized_images, labels) if label == 'OSCC images']

# Count the number of existing images for each class
num_normal_images = len(normal_images)
num_oscc_images = len(oscc_images)

# Upsample the classes to match the number of synthetic images per class
while num_normal_images < num_synthetic_images_per_class:
    image_data = random.choice(normal_images)
    synthetic_image, synthetic_label = generate_synthetic_image(image_data, 'Normal')
    normal_images.append(synthetic_image)
    num_normal_images += 1

while num_oscc_images < num_synthetic_images_per_class:
    image_data = random.choice(oscc_images)
    synthetic_image, synthetic_label = generate_synthetic_image(image_data, 'OSCC')
    oscc_images.append(synthetic_image)
    num_oscc_images += 1


# Sample 224 images randomly from each class
selected_normal_images = random.sample(normal_images, num_synthetic_images_per_class)
selected_oscc_images = random.sample(oscc_images, num_synthetic_images_per_class)

# Combine images and labels to create the final dataset
synthetic_images = (
    selected_normal_images +
    selected_oscc_images
)

synthetic_labels = (
    ['Normal images'] * num_synthetic_images_per_class +
    ['OSCC images'] * num_synthetic_images_per_class
)

print("Synthetic Images are generated by flipping & rotation, stored as synthetic_images and synthetic_labels")


In [ ]:
#counting the number of labels in each classes

count_of_classes = {}

for label in synthetic_labels:
    if label in count_of_classes:
        count_of_classes[label] +=1
    else:
        count_of_classes[label] = 1


for key, value in count_of_classes.items():
    print(f'{key}:{value}')

In [ ]:
# Function to display images
def plot_images(original_images, generated_images, num_images=1000):
    fig, axes = plt.subplots(num_images, 2, figsize=(8, 2*num_images))
    for i in range(num_images):
        axes[i, 0].imshow(original_images[i])
        axes[i, 0].set_title('Original')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(generated_images[i])
        axes[i, 1].set_title('Generated')
        axes[i, 1].axis('off')
    plt.tight_layout()
    plt.show()

# Display original and generated images side by side
num_images_to_display = 5
original_images_to_display = normalized_images[:num_images_to_display]
generated_images_to_display = synthetic_images[:num_images_to_display]

plot_images(original_images_to_display, generated_images_to_display, num_images=num_images_to_display)

#### Balanced number of images/class

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:

# Initialize empty lists to store processed images and corresponding labels
processed_images = []
processed_labels = []

# Load, preprocess, and align images and labels
for image, label in zip(synthetic_images, synthetic_labels):
    try:
        processed_images.append(image)
        processed_labels.append(label)
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")

# Convert labels to one-hot encoding
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(processed_labels)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded_labels = integer_encoded_labels.reshape(len(integer_encoded_labels), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded_labels)

# Convert processed_images and onehot_encoded_labels to numpy arrays
processed_images = np.array(processed_images)
onehot_encoded_labels = np.array(onehot_encoded_labels)

# Save processed_images and onehot_encoded_labels in the current directory
np.save("processed_images.npy", processed_images)
np.save("onehot_encoded_labels.npy", onehot_encoded_labels)

# Verify the shapes of processed_images and onehot_encoded_labels
print("Shape of processed_images:", processed_images.shape)
print("Shape of onehot_encoded_labels:", onehot_encoded_labels.shape)

# Model Development

In [ ]:
# Import ResNeXt50 from tensorflow.keras.applications
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, MobileNetV2, Xception, InceptionResNetV2, EfficientNetB0

In [ ]:
#importing the model & dense layer for customizing the neural network
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [ ]:
#input shape & dimension for the pre-trained models
shape=(224, 224, 3)

In [ ]:
# Load pre-trained models
base_model_1 = VGG16(weights='imagenet', include_top=False, input_shape=shape)
base_model_2 = VGG19(weights='imagenet', include_top=False, input_shape=shape)
base_model_3 = ResNet50(weights='imagenet', include_top=False, input_shape=shape)
base_model_4 = InceptionV3(weights='imagenet', include_top=False, input_shape=shape)
base_model_5 = DenseNet121(weights='imagenet', include_top=False, input_shape=shape)
base_model_6 = MobileNetV2(weights='imagenet', include_top=False, input_shape=shape)
base_model_7 = Xception(weights='imagenet', include_top=False, input_shape=shape)
base_model_8 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=shape)
#base_model_9 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=shape)

print('Loaded')

In [ ]:
base_models = [base_model_1, base_model_2, base_model_3,
               base_model_4, base_model_5, base_model_6,
               base_model_7, base_model_8]

In [ ]:
# Looping through the Base models and printing the summaries
for idx, model in enumerate(base_models):
    print(f'Summary of Base Model {idx +1}:')
    model.summary()

In [ ]:
#Freezing the pre-trained model's last layer for transfer learning
for model in base_models:
    for layer in model.layers:
        layer.trainable=False

## Customize the model

In [ ]:
custom_models = []

for idx, model in enumerate(base_models):
    x = model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    #x = Dense(512, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)
    custom_model = Model(inputs=model.input, outputs=predictions)
    custom_models.append(custom_model)
    print(f"Customized the model - {idx+1}")

In [ ]:
# Looping through the Base models and printing the summaries
for idx, model in enumerate(custom_models):
    print(f'Summary of Custom Model {idx +1}:')
    model.summary()

## Compile the model

In [ ]:
compiled_models = []
for idx, custom_model in enumerate(custom_models):
    custom_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    compiled_models.append(custom_model)
    print(f"Compiled Custom Model {idx + 1}")

In [ ]:
for idx, model in enumerate(compiled_models):
    print(f'Summary of compiled model {idx +1}: ')
    model.summary()

## Train the model

In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and temporary sets (60% training, 40% temporary)
X_train, X_temp, y_train, y_temp = train_test_split(processed_images, onehot_encoded_labels, test_size=0.3, random_state=42)

# Split the temporary data into testing and validation sets (50% testing, 50% validation)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Verify the shapes of the training, testing, and validation sets
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

def train_and_evaluate_models(X_train, y_train, X_test, y_test, compiled_models, model_names, epochs=100):
    results = []
    for model, model_name in zip(compiled_models, model_names):
        # Initialize variables for tracking maximum accuracy and epoch
        max_accuracy = 0
        max_accuracy_epoch = 0

        # Define a checkpoint to save the model when target accuracy is reached
        checkpoint = ModelCheckpoint(f'{model_name}_model.h5', monitor='val_accuracy',
                                     save_best_only=True, save_weights_only=False, mode='max', verbose=1)

        # Define early stopping to stop training if accuracy doesn't improve for 10 epochs
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)

        # Define CSV logger to save training history
        csv_logger = CSVLogger(f'{model_name}_training_history.csv', append=True)

        # Train the current model
        history = model.fit(
            X_train, y_train,
            epochs=epochs,
            validation_data=(X_test, y_test),
            batch_size=32,
            callbacks=[checkpoint, early_stopping, csv_logger]
        )

        # Get the epoch at which maximum accuracy is obtained
        max_accuracy_epoch = np.argmax(history.history['val_accuracy'])
        max_accuracy = history.history['val_accuracy'][max_accuracy_epoch]

        # Evaluate the model on the test data
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f'Test Accuracy for {model_name}: {test_accuracy}')

        # Store results including the epoch
        results.append({'Model': model_name, 'Accuracy': max_accuracy, 'Epoch': max_accuracy_epoch+1})

        print("-" * 40)  # Print a line of dashes
        print(f"Maximum accuracy of {max_accuracy:.2f} achieved at epoch {max_accuracy_epoch+1}")
        print("Model with high accuracy is saved using the keras ModelCheckpoint")
        print("-" * 40)  # Print a line of dashes
        print("\n")

        # Assuming model.predict returns probabilities for each class
        y_pred_probs = model.predict(X_test)

        # Convert probabilities to class labels
        y_pred = np.argmax(y_pred_probs, axis=1)

        # Convert true labels to class labels if y_test is one-hot encoded
        y_true = np.argmax(y_test, axis=1)

        # Generate confusion matrix
        conf_matrix = confusion_matrix(y_true, y_pred)

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(model_name)
        plt.show()
    return results

In [ ]:
print(count_of_classes)

In [ ]:
model_names = ["VGG16", "VGG19", "ResNet50", "InceptionV3", "DenseNet121", "MobileNetV2", "Xception", "InceptionResNetV2"]
class_labels = ['Normal', 'OSCC']

In [ ]:
#Train and Evaluate the model
results = train_and_evaluate_models(X_train, y_train, X_test, y_test, compiled_models, model_names, epochs=100)

In [ ]:
print(results) #print the results.

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
from tabulate import tabulate

# Define the headers for the table
headers = ["Model", "Epoch"]

# Print the table
print('Models Accuracy')
print(tabulate(results_df, headers, tablefmt="grid"))

In [ ]:
#prediction

In [ ]:
os.getcwd()

In [ ]:
import os
import tensorflow as tf

directory_path = '/tf/red-model/added models'
loaded_models = {}

for filename in os.listdir(directory_path):
    if filename.endswith(".h5"):  # Assuming the models are saved in HDF5 format with .h5 extension
        model_name = os.path.splitext(filename)[0]  # Get the model name without extension
        model_path = os.path.join(directory_path, filename)
        loaded_model = tf.keras.models.load_model(model_path)  # Load the model using TensorFlow
        loaded_models[model_name] = loaded_model

# Print the loaded model names
print("Loaded model names:")
print(list(loaded_models.keys()))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

for model_name, model in loaded_models.items():
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Calculate accuracy score
    accuracy = accuracy_score(y_true, y_pred)

    # Generate classification report
    class_report = classification_report(y_true, y_pred, target_names=class_labels)

    # Print individual class prediction accuracy score
    print(f"Accuracy for {model_name}: {accuracy:.2f}")
    print(f"Classification Report for {model_name}:\n{class_report}")


In [ ]:
from sklearn.metrics import accuracy_score

for model_name, model in loaded_models.items():
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Calculate accuracy score
    accuracy = accuracy_score(y_true, y_pred)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f"{model_name} - Accuracy: {accuracy:.2f}")  # Include accuracy in the title
    plt.show()

    # Print accuracy score
    print(f"Accuracy for {model_name}: {accuracy:.2f}")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

for model_name, model in loaded_models.items():
    y_pred_probs = model.predict(X_val)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_val, axis=1)

    # Calculate accuracy score
    accuracy = accuracy_score(y_true, y_pred)

    # Generate classification report
    class_report = classification_report(y_true, y_pred, target_names=class_labels)

    # Print individual class prediction accuracy score
    print(f"Accuracy for {model_name}: {accuracy:.2f}")
    print(f"Classification Report for {model_name}:\n{class_report}")

In [ ]:
from sklearn.metrics import accuracy_score

for model_name, model in loaded_models.items():
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Calculate accuracy score
    accuracy = accuracy_score(y_true, y_pred)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f"{model_name} - Accuracy: {accuracy:.2f}")  # Include accuracy in the title
    plt.show()

    # Print accuracy score
    print(f"Accuracy for {model_name}: {accuracy:.2f}")

In [ ]:
# test the model

In [ ]:
import os
from keras.models import load_model

# Define class labels and their corresponding encoded values

# Load the pre-trained models
model_folder = '/tf/Purushothaman/6 Class model/Balanced Model'
# Replace with the path to your model folder
model_files = os.listdir(model_folder)
loaded_models = {}

for model_file in model_files:
    try:
        model_name, extension = os.path.splitext(model_file)
        if extension == '.h5':
            model_path = os.path.join(model_folder, model_file)
            loaded_models[model_name] = load_model(model_path)
            print(f"Loaded model: {model_name}")
        else:
            print(f"Skipped file: {model_file} (not a valid model file)")
    except Exception as e:
        print(f"Error loading model {model_file}: {str(e)}")

print("Model loading completed.")

In [ ]:
# Load label encoder classes
label_encoder_classes = np.load(os.path.join(model_folder, 'onehot_encoded_labels.npy'))

# Set the classes_ attribute of the LabelEncoder
label_encoder = LabelEncoder()
label_encoder.classes_ = label_encoder_classes
print('loaded')

In [ ]:
class_labels = ['Normal images', 'OSCC images']
label_encoder.fit(class_labels)

In [ ]:
from PIL import Image
import os
import numpy as np
import pandas as pd

# Define the folder containing images
image_folder = "/tf/Test images/"

# List all files in the folder
image_files = os.listdir(image_folder)

# Define the target size for resizing
target_size = (224, 224)

# Define a list to store the results
image_results = []

# Iterate through each image file
for image_file in image_files:
    # Get the full file path
    image_path = os.path.join(image_folder, image_file)

    try:
        # Open and process the image using PIL (Pillow)
        pil_image = Image.open(image_path)

        # Resize and convert image to RGB format if necessary
        pil_image = pil_image.resize(target_size)
        if pil_image.mode != 'RGB':
            pil_image = pil_image.convert('RGB')

        # Convert image to numpy array
        numpy_image = np.array(pil_image)

        # Normalize image to [0, 1]
        normalized_image = numpy_image.astype(np.float32) / 255.0

        # Add batch dimension
        normalized_image = np.expand_dims(normalized_image, axis=0)

        image_result = {"Image": image_file}

        # Iterate through loaded models and make predictions
        for model_name, model in loaded_models.items():
            # Predict using the current model
            predictions = model.predict(normalized_image)

            # Convert numerical predictions to class labels using inverse_transform
            predicted_classes = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

            # Add model predictions to the dictionary
            image_result[model_name] = predicted_classes[0]  # Store decoded predictions for each model

        # Add the dictionary to the results list
        image_results.append(image_result)
    except Exception as e:
        print(f"Error processing image {image_file}: {str(e)}")

# Create a DataFrame from the results list
df = pd.DataFrame(image_results)

# Print the DataFrame with model names as columns
print(df)


In [ ]:
# Export the DataFrame to a CSV file
df.to_csv('output.csv', index=False)

print("DataFrame has been exported to 'output.csv'")

In [ ]:
os.getcwd()

In [ ]:
df = pd.read_csv('/tf/Purushothaman/6 Class model/Balanced Model/output.csv')

In [ ]:
df.T